# Assignment 2: Linear regression

## Marking Breakdown

**70-100%** results/answer correct plus extra achievement at understanding or analysis of results. Clear explanations, evidence of creative or deeper thought will contribute to a higher grade.

**60-69%** results/answer correct or nearly correct and well explained.

**50-59%** results/answer in right direction but significant errors.

**40-49%** some evidence that the student has gained some understanding, but not answered the questions
properly.

**0-39%** serious error or slack work.

## Imports

In [ ]:
from __future__ import print_function, division
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.metrics import r2_score, explained_variance_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import train_test_split, KFold, cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import OneHotEncoder, LabelEncoder

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
import seaborn as sns
import math
import matplotlib.patches as mpatches
%matplotlib inline

## Description of the dataset
This assignment is based on the automobile pricing dataset. Our goal will be to predict the price of automobiles based on various attributes. This data set consists of three types of entities: 

1. The specification of an automobile in terms of various characteristics 

1. Assigned insurance risk rating 
   * this rating corresponds to the degree to which the auto is more risky than its price indicates. Cars are initially assigned a risk factor symbol associated with its price. Then, if it is more risky (or less), this symbol is adjusted by moving it up (or down) the scale. Actuaries call this process ”symboling”. A value of +3 indicates that the auto is risky, -3 that it is probably pretty safe. 

1. Normalized losses in use as compared to other cars
  * the third factor is the relative average loss payment per insured vehicle year. This value is normalized for all autos within a particular size classification (two door small, station wagons, sports/speciality, etc...), and represents the average loss per car per year (avg_loss/car/year). 


To save you time and to make the problem manageable with limited computational resources, we preprocessed the original dataset. We removed any instances that had one or more missing values and randomized the data set. The resulting representation is much more compact and can be used directly to perform our experiments.


## 1. Simple Linear Regression [50%]
We will begin by studying a simple Linear Regression model. Such a model will consider the relationship between a dependent (response) variable and only one independent (explanatory) variable. When applying machine learning in practice it can be prudent to start out simple in order to get a feeling for the dataset and for any potential difficulties that might warrant a more sophisticated model. In this Section we will consider one independent variable (i.e. feature) `engine-power` against the dependent variable (i.e. target) `price`.

### ========== Question 1.1 --- [1 mark] ==========
Load the dataset `train_auto_numeric.csv` into a pandas DataFrame called `auto_numeric`. Display the number of data points and attributes in the dataset.

In [ ]:
data_path = os.path.join(os.getcwd(), 'data', 'train_auto_numeric.csv')
auto_numeric = pd.read_csv(data_path, delimiter = ',')
print('Number of data points: {}, number of attributes: {}'.format(auto_numeric.shape[0], auto_numeric.shape[1]))

### ========== Question 1.2 --- [1 mark] ==========
Display the first 8 instances of the dataset.

In [ ]:
auto_numeric.head(8)

### ========== Question 1.3 --- [1 mark] ==========
Display the summary statistics for the dataset.

In [ ]:
auto_numeric.describe()

### ========== Question 1.4 --- [2 marks] ==========
Produce a scatter plot of `price` against `engine-power`. Label axes appropriately and include a title in your plot.

In [ ]:
def scatter_jitter(arr1, arr2, jitter=0.1):
    """ Plots a joint scatter plot of two arrays by adding small noise to each example. 
    Noise is proportional to variance in each dimension. """
    arr1 = np.asarray(arr1)
    arr2 = np.asarray(arr2)
    arr1 = arr1 + jitter*arr1.std(axis=0)*np.random.standard_normal(arr1.shape)
    arr2 = arr2 + jitter*arr2.std(axis=0)*np.random.standard_normal(arr2.shape)
    plt.scatter(arr1, arr2, c='g', alpha=0.5)

scatter_jitter(auto_numeric["engine-power"], auto_numeric["price"])
plt.xlabel("Engine power")
plt.ylabel("Price")
plt.title("Price vs. Engine power")
print()

### ========== Question 1.5 --- [2 marks] ==========
Do you think that engine-power alone is sufficient for predicting the price? Can you make any other observations on the data from the above plot? Please explain your answer in 2-3 sentences. 

*I don't think that engine-power alone is sufficient, as it is not enough correlated with the price. There are quite a few data points where the engine power is high and the price low, or vice versa. However, some of such examples are also very far away from the other data points, could be possibly considered to be outliers.*

### ========== Question 1.6 --- [2 marks] ==========
Visualise the distribution of the car prices. Choose a sensible value for the number of bins in the histogram. Again, label axes appropriately and include a title in your plot.

In [ ]:
# the histogram of the data
n, bins, patches = plt.hist(auto_numeric["price"], 15, facecolor='g', alpha=0.75)
plt.xlabel('Price')
plt.ylabel('Count')
plt.title('Histogram of Price')
plt.grid(True) # Enables grid
plt.show()

### ========== Question 1.7 --- [2 marks] ==========
How could you preprocess the data to improve the performance of linear regression? Don’t do it at this stage, but instead in one sentence explain why you would do what you suggested.

*We may check scale the features, and/or remove the outliers if there are any (some outliers might be simply very expensive cars).*

### ========== Question 1.8 --- [1 mark] ==========
Now we want to build a simple linear regression model. First we need to define our input and target variables. Store the values of the attribute `engine-power` in a vector `X` and the values of our target variable `price` in a vector `y`.

In [ ]:
X = np.array(auto_numeric["engine-power"])
y = np.array(auto_numeric["price"])

### ========== Question 1.9 --- [1 mark] ==========
For technical reasons, we need to convert `X` into a 2D array, otherwise we will receive an error when trying to use it for building models. Perform this transformation and confirm that the shape of the resulting array is (`n`,1) where `n` is the number of instances in the dataset. 

In [ ]:
X = X.reshape((X.size, 1))
print (X.shape)

### ========== Question 1.10 --- [1 mark] ==========
Now we want to use Hold-out validation to split the dataset into training and testing subsets. Use 80% of the data for training and the remaining 20% for testing. Store your data into matrices `X_train`, `X_test`, `y_train`, `y_test`.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)
print (X_train.shape, X_test.shape, y_train.shape, y_test.shape)

### ========== Question 1.11 --- [2 marks] ==========
By using Scikit-learn's [`LinearRegression`](http://scikit-learn.org/stable/modules/generated/sklearn.linear_model.LinearRegression.html) fit a model to the training data. When initialising the model, set the `normalize` parameter to `True` and use default settings for the other parameters.

In [ ]:
regressor = LinearRegression(normalize=True).fit(X_train, y_train)
regressor

### ========== Question 1.12 --- [2 marks] ==========
By looking into the attributes of your model, write down an equation for predicting the price of a car given the engine-power.

In [ ]:
print('Coefficient: ', regressor.coef_)
print('Intercept: ', regressor.intercept_)

def car_price(model, engine_power): 
    return (model.coef_ * engine_power + model.intercept)

# The equation is of form y = ax + b, where a is the coefficient, and b the intercept, thus:
# car price = 0.08988389 * engine power + 2823.1218911    

### ========== Question 1.13 --- [3 marks] ==========
What happens to the price as one more unit of engine-power is added? By examining the magnitude of the regression coefficient is it possible to tell whether or not engine-power is an important influential variable on price? Explain your answer in 1-2 sentences.

*If one more unit of engine-power is added, the price goes up by 0.08988389. The magnitude of the regression coefficient is small, which may imply that engine-power is not a very influential variable on price, but this may not necessarily be truth, as engine-power has a very large std (30k) so this small coefficient gets multiplied with very large values and then added to the price.*

### ========== Question 1.14 --- [2 marks] ==========
Produce a scatter plot similar to the one in Question 1.4 but use training data only this time. Add the regression line to the plot and show the predictions on the training set by using a different marker. Label axes appropriately and add a title to the graph. 

In [ ]:
# Make predictions using the testing set
training_predictions = regressor.predict(X_train)
scatter_jitter(X_train, y_train, jitter=0)
plt.plot(X_train, training_predictions, color='r', linewidth=2)
plt.xlabel("Engine power")
plt.ylabel("Price")
plt.title("Engine power vs. Price")
red_patch = mpatches.Patch(color='red', label='Linear regression model predictions')
blue_patch = mpatches.Patch(color='green', label='Training data')
plt.legend(handles=[red_patch, blue_patch])
print()

### ========== Question 1.15 --- [2 marks] ==========
So far we have used Hold-out validation. Can you think of a disadvantage of using this method, especially when dealing with small datasets?

*The main disadvantage of Hold-out validation is high variance of the model evaluation results. The model is fit and tested only once, thus, it's predictive abilities highly depend on how the dataset was split into training/testing subsets. If the dataset is small, or the variance of variable values is high, it is even more likely that the random split would result in a model that generalizes very poorly.*

### ========== Question 1.16 --- [1 mark] ==========
Now we want to use k-fold cross-validation to evaluate the performance of the regression model. Famliriase yourself with the sklearn method [`KFold`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) and make sure you understand the differences between Hold-out and K-fold cross-validation. By using Scikit-learn's [`KFold`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.KFold.html) class construct a 5-fold cross-validation object. Set the `shuffle` parameter to `True` and `random_state` to `0`. Use the object to print the training and validation indices for the `auto_numeric` dataset (hint: see the `split` method).

In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=0)
counter = 0
for train, test in kf.split(auto_numeric):
    counter += 1
    print ("\nFold number: %s"%counter)
    print ("Training indices: %s"%train)
    print ("Validation indices: %s"%test)

### ========== Question 1.17 --- [3 marks] ==========
By making use of the iterator you constructed in the previous question, loop through the 5 folds and display the mean value of the `price` variable for the training instances in each fold.

In [ ]:
for train, test in kf.split(auto_numeric): 
    print ("The mean value of the price variable in the fold: %s"%auto_numeric.iloc[train].price.mean())

### ========== Question 1.18 --- [3 marks] ==========
Now initialise a new `LinearRegression` model and fit it by making use of the cross-validation iterator, the `X` and `y` arrays defined above and the [`cross_val_predict`](http://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_predict.html) function. Display the shape of your prediction and confirm it has the same dimensionality as your `y` vector.

In [ ]:
model = LinearRegression(normalize=True)
predictions = cross_val_predict(model, X, y, cv=kf)
print ("y shape: %s\npredictions shape: %s"%(y.shape, predictions.shape))

### ========== Question 1.19 --- [2 marks] ==========
Report the Coefficient of Determination (R^2), Root Mean Squared Error (RMSE), Mean Absolute Error (MAE) and Correlation Coefficient (CC) from the simple linear regression model you build in Question 1.18. *Hint: RMSE is the square root of the Mean Squared Error (MSE). For CC you might find numpy's [`corrcoef`](http://docs.scipy.org/doc/numpy/reference/generated/numpy.corrcoef.html) function useful.* 

In [ ]:
print ("Coefficient of Determination: %s"%r2_score(y, predictions))
print ("Root Mean Squared Error: %s"%math.sqrt(mean_squared_error(y, predictions)))
print ("Mean Absolute Error: %s"%mean_absolute_error(y, predictions))
print ("Correlation Coefficient: %s"%np.corrcoef(y, predictions, rowvar = False)[0, 1])

### ========== Question 1.20 --- [4 marks] ==========
What do the above metrics intend to measure? Relate the values of CC, MAE and RMSE to the observations you made in Question 1.5. Explain your answer in 1-2 short paragraphs.

* *RMSE - the standard deviation of model prediction errors (residuals). The errors are measured in terms of how far the values predicted by the model are from the true values. RMSE tells how the data is spread around the best fit mark.* 
* *MAE - the mean of all absolute model prediction errors (residuals).*
* *CC - measures how related are two variables. Calculated dividing the covariance of the variables by the product of their standard deviations.*
 
*If the dataset is small and/or hold-out validation is used, the training set attribute values are more likely to be not normally distributed and deviating a lot. Consequently, the model will generalize (when testing on new data) poorly, and the RMSE/MAE, will be high. Furthermore, high standard deviations of the variables will affect correlation coefficient, making it lower.*

### ========== Question 1.21 --- [3 marks] ==========
Show a histogram of the residuals of the linear regression model (i.e. true - predicted values). Label axes appropriately and add a title to your plot. Does the distribution of residuals look like what you expected?

In [ ]:
n, bins, patches = plt.hist((y - predictions), 15, facecolor='g', alpha=0.75)
plt.xlabel('Residual')
plt.ylabel('Count')
plt.title('Histogram of Residuals')
plt.grid(True)
plt.show()

*The distribution looks somewhat as expected: most of the residuals fall within a small range nearby 0 (meaning that there were a lot of small errors made by the model), and others are more or less normally distributed around that. There are some extremely high and low values, which must be related to the outlier (the errors are huge, because the outliers have significantly different values from the other data points). Interestingly, much bigger part of the residuals are below 0, implying that the model overestimated (predicted higher than true) the value more often.*

### ========== Question 1.22 --- [2 marks] ==========
Load the new dataset `train_auto_base.csv` into a pandas DataFrame `auto_base`. Again by using the `engine-power` attribute as predictor and `price` as target variable build a LinearRegression model on this dataset. Report the R^2, RMSE, MAE and CC metrics for this model by making use of the K-fold CV iterator constructed in Question 1.16.

In [ ]:
data_path = os.path.join(os.getcwd(), 'data', 'train_auto_base.csv')
auto_base = pd.read_csv(data_path, delimiter = ',')

X = auto_base["engine-power"].values
y = auto_base["price"].values
X = X.reshape(X.size, 1)
y = y.reshape(y.size, 1)

model_new = LinearRegression(normalize=True)
predictions = cross_val_predict(model_new, X, y, cv=kf)

print ("Coefficient of Determination: %s"%r2_score(y, predictions))
print ("Root Mean Squared Error: %s"%math.sqrt(mean_squared_error(y, predictions)))
print ("Mean Absolute Error: %s"%mean_absolute_error(y, predictions))
print ("Correlation Coefficient: %s"%np.corrcoef(y, predictions, rowvar = False)[0, 1])

### ========== Question 1.23 --- [2 marks] ==========
Show a scatter plot of predicted vs. true prices and another one of predicted price vs. engine-power. Use a single plot with two subplots. Label axes appropriately.

In [ ]:
def fit_scatter(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    fig, ax = plt.subplots()
    ax.scatter(y_true, y_pred)
    ax.plot([y_true.min(), y_true.max()], [y_true.min(), y_true.max()], 'k--', lw=4)
    ax.set_xlabel('Measured')
    ax.set_ylabel('Predicted')

# Single plot with two subplots (as requested).
auto_base['predictions'] = predictions.flatten().tolist()
extra = sns.pairplot(data=auto_base, x_vars=['engine-power', 'price'], y_vars = ['predictions'], height = 5)


### ========== Question 1.24 --- [3 marks] ==========
What is the simplest baseline model for the purposes of regression? Relate your answer to the regression model you have just built as part of this question. Can the predictions of this model be justified given the procedure you followed to train it?

*The simplest baseline model would be calculating the mean value of the dependent variable for predictions (here, price). The model built above is doing precisely that, because the independent engine-power variable is 1 for every data point, and the model relies only on target price averages for predictions. Given the procedure, model predictions are justified to some extent: it predicts averages, but due to cross-validation there are 5 of them.*

### ========== Question 1.25 --- [2 marks] ==========
Why do you think this model performs so poorly? (*Hint: Justify your answer by displaying some statistics about the `auto_base` dataset.*)

In [ ]:
auto_base.price.describe()

*The predictions are based on the average of the dependent variable values, though these have a high standard deviation, are not normally distributed, and still include outliers. Consequently, the model errors are big and the performance low.*

## 2. Multivariate Linear Regression [50%]
In this Section we will fit a Multivariate Linear Regression model (LinearRegression) to the dataset. In contrast to Part 1, we will now train a model with multiple explanatory variables and ascertain how they affect our ability to predict the retail price of a car. One of our foremost concerns will be to determine exactly which attributes to include in the model and which may be left out.

### ========== Question 2.1 --- [10 marks] ==========
Use the original dataset (`auto_numeric`) and a visualisation tool of your choice to examine whether or not any of the other attributes are particularly good at predicting the price. Can you find any? Do any attributes appear useless at predicting the price? Do any attributes exhibit significant correlations? As you answer these questions, list two attributes for each question but do not modify the dataset at this stage. Of the attributes you listed, which ones could you safely remove? Explain in 4-5 sentences. *Hint: you might find seaborn's [`pairplot`](https://seaborn.github.io/generated/seaborn.pairplot.html?highlight=pairplot#seaborn.pairplot) function useful for this question.*

In [ ]:
for x in list(auto_numeric)[:-1]: 
    cc = np.corrcoef(auto_numeric[x], auto_numeric.price)[0,1].round(5)
    pl = sns.pairplot(data=auto_numeric, x_vars=x, y_vars = 'price', height=6)
    pl.fig.suptitle(f'CC = {cc}', y=0.01)
    print ("Correlation Coefficient between price and %s: %s"%(x, cc))

*Inspecting the pairplots for all possible combinations of independent/dependent variables, as well as calculating the correlation coefficient between these pairs, allowed to make to following conclusions:*

* *`engine-size` seems to be particularly good at predicting the price, since the distribution of the data points in the pairplot seems linear, and the values have a small variance (all data points are somewhat tightly packed). `length` and `width` are also pretty good, showing same effect as engine-size, but in a smaller scale.*

* *2 attributes that look most useless are `normalized-losses` and `peak-rpm` (also `mean-effective-pressure`), having pairplots that seem most randomly distributed. What is more, their correlation coefficients with the price are the lowest from all attributes.* 

* *significant correlations are exhibited by these attribues (from highest to lowest): `engine-size`, `width`, `length`, `engine-power`, `highway-mpg`, `wheel-base`.*

* *Of the listed attributes, `normalized-losses` and `peak-rpm` could be safely removed, since the relationship between these attributes and price is non-linear, and they do not exhibit significant correlations with the thing that we want to predict.* 

### ========== Question 2.2 --- [3 marks] ==========
We will now make a first attempt at building a Multivariate Linear Regression model using all numeric attributes. Initialise a `LinearRegression` model and predict the output by using 5-fold cross-validation and the `cross_val_predict` function.

In [ ]:
X = np.array(auto_numeric.drop(['price'], axis=1))
y = np.array(auto_numeric["price"])
y = y.reshape(y.size, 1)
multivariate = LinearRegression()
predictions = cross_val_predict(multivariate, X, y, cv=kf)

### ========== Question 2.3 --- [2 marks] ==========
Display the Root Mean Squared Error (RMSE), Mean Absolute Error (MAE) and Correlation Coefficient (CC).

In [ ]:
print ("Root Mean Squared Error: %s"%math.sqrt(mean_squared_error(y, predictions)))
print ("Mean Absolute Error: %s"%mean_absolute_error(y, predictions))
print ("Correlation Coefficient: %s"%np.corrcoef(y, predictions, rowvar = False)[0, 1])

### ========== Question 2.4 --- [2 marks] ==========
 Comment on each metric display above in comparison to what you have obtained for the Simple Linear Regression model in Question 1.19.


* RMSE for the multivariate model is more than 20% lower than for the simple model (4797 vs. 6114).
* MAE is around 20% lower (3271 vs 3987)
* CC between model predictions and true prices is almost twice bigger for the multivariate model (0.735 vs 0.417).

*To sum up, the multivariate model made smaller and less deviating errors, and the predictions made by it were in correlation with the actual data much more than for the simple linear regression model.*

### ========== Question 2.5 --- [2 marks] ==========
Examine the histogram for the `engine-size` attribute. Choose a sensible value for the number of bins in the histogram. Label axes appropriately and include a title in your plot.

In [ ]:
n, bins, patches = plt.hist(auto_numeric['engine-size'], 10, facecolor='g', alpha=0.75)
plt.xlabel('Engine size')
plt.ylabel('Count')
plt.title('Histogram of engine sizes')
plt.grid(True)
plt.show()

### ========== Question 2.6 --- [2 marks] ==========
Is the distribution expected to cause a problem for regression? Explain your answer in 2-3 sentences.

*Generally, linear regression does not make any assumptions about the independent variable, so the distribution of it does not really matter so much. However, it can be observed that most of the values in the histogram fall within a very small range (0-20) and there are not that many values above (apart from the outliers). That could imply that the relationship between the IV and DV might be non-linear, causing problems for regression.*

### ========== Question 2.7 --- [3 marks] ==========
Transform this attribute using an appropriate simple technique from the lectures. Plot the histogram of the transformed attribute.

In [ ]:
transformed = np.log(auto_numeric['engine-size'])
n, bins, patches = plt.hist(transformed, 10, facecolor='g', alpha=0.75)
plt.xlabel('Engine size')
plt.ylabel('Count')
plt.title('Histogram of engine sizes')
plt.grid(True)
plt.show()

### ========== Question 2.8 --- [3 marks] ==========
Now re-build a Linear Regression model on the transformed dataset and report the R^2, RMSE, MAE and CC metrics.

In [ ]:
transformed = auto_numeric.copy(deep=True)
transformed['engine-size'] = np.log(transformed['engine-size'])
X = np.array(transformed.drop(['price'], axis=1))
y = np.array(transformed["price"])
y = y.reshape(y.size, 1)

model = LinearRegression(normalize=True)
predictions = cross_val_predict(model, X, y, cv=5)

print ("Coefficient of Determination: %s"%r2_score(y, predictions))
print ("Root Mean Squared Error: %s"%math.sqrt(mean_squared_error(y, predictions)))
print ("Mean Absolute Error: %s"%mean_absolute_error(y, predictions))
print ("Correlation Coefficient: %s"%np.corrcoef(y, predictions, rowvar = False)[0, 1])

### ========== Question 2.9 --- [3 marks] ==========
How has the performance of your model changed? Explain your answer in 1-2 sentences.

*The errors got smaller, and the correlation coefficient got higher. Transforming the variable made the relationship between it and the dependant variable (price) more linear. Consequently, model predictions were more accurate.*

### ========== Question 2.10 --- [2 marks] ==========
So far we have performed regression with numeric attributes. We will now attempt to integrate nominal (categorical) attributes into our regression model. 
Load the dataset `train_auto_full.csv` into a pandas DataFrame called `auto_full`. Display the number of samples and attributes in the dataset. Also, display the first 20 instances of the dataset. *Hint: Execute the cell below to change the default for `max_columns` display option in pandas.*

In [ ]:
pd.set_option('max_columns', 30)

In [ ]:
data_path = os.path.join(os.getcwd(), 'data', 'train_auto_full.csv')
auto_full = pd.read_csv(data_path, delimiter = ',')
print('Number of data points: {}, number of attributes: {}'.format(auto_full.shape[0], auto_full.shape[1]))
auto_full.head(20)

### ========== Question 2.11 --- [3 marks] ==========
This dataset contains a mixture of numeric and nominal attributes. Name the variables that you think are categorical. Why can we not use the nominal attributes in their current form for the purposes of regression? 

*The categorical variables are: `make`, `fuel-type`, `aspiration`, `num-of-doors`, `body-style`, `drive-wheels`, `engine-location`, `engine-type`, `num-of-cylinders`, `fuel-system`, `symboling`. We can't use nominal attributes in their current form for the purposes of regression, because it requires attributes to have continuous values. Specifically, regression involves arithmetic operations that can't be performed on (non-quantified) nominal variables.*

### ========== Question 2.12 --- [5 marks] ==========
Now we want to convert the categorical variables by using [One-Hot-Encoding](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder). Familiarise yourself with the class. One limitation with this module is that it can deal only with categorical attributes in integer format (remember that in our example we have attributes in string format). 

Copy the `auto_full` dataframe into a new dataframe `auto_full_edit` and transform the categorical variables by using [Label Encoding](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.LabelEncoder.html). Then transform again the categorical variables by using One-Hot-Encoding. Make sure you don't transform the continuous variables. *Hint: make appropriate use of the `categorical_features` parameter in [`OneHotEncoder`](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html#sklearn.preprocessing.OneHotEncoder).*

Store the transformed attributes into a numpy array `X_enc` and display its dimensionality.  


In [ ]:
# Copy the dataset and list names of columns that have categorical/ordinal attributes.
auto_full_edit = auto_full.copy(deep=True)
cat_cols = ['make', 'fuel-type', 'aspiration', 'num-of-doors', 'body-style', 'drive-wheels', 'engine-location', 'engine-type', 'num-of-cylinders', 'fuel-system', 'symboling']
idx = np.array([auto_full_edit.columns.get_loc(col_name) for col_name in cat_cols])

# Encoding categorical data as integers:
auto_full_edit[cat_cols] = auto_full_edit[cat_cols].apply(LabelEncoder().fit_transform)

# Removing the targets' column (prices), because one-hot encoding procedure will change the ordering of the data and
# output a numpy array without the labels (this column will not be necessary for the following calculations anyway). 
auto_full_edit = auto_full_edit.drop(['price'], axis = 1)

# Mask: boolean array of n_features length (True for categorical columns, False otherwise). 
cat_idx = np.array([auto_full_edit.columns.get_loc(col_name) for col_name in cat_cols])
mask = np.zeros(auto_full_edit.shape[1], dtype=bool)
mask[cat_idx] = True

# One-hot encoding the categorical attributes in the dataset (using a mask).
X_enc = OneHotEncoder(categorical_features=mask).fit_transform(auto_full_edit).toarray()
print('Dimensions of the X_enc array with transformed attributes and removed "price" column: {}'.format(X_enc.shape))

### ========== Question 2.13 --- [2 marks] ==========
By using the transformed data train a multivariate linear regression model and by using 5-fold cross-validation report the R^2, RMSE, MAE and CC metrics.

In [ ]:
# In multivariate linear regression, the ordering of the attributes matter, and OneHotEncoder
# stacks the non-categorical ones to the end of the array. To bring more important features 
# upfront, X_enc is flipped.
X = np.fliplr(X_enc)
y = np.array(auto_full["price"])
y = y.reshape(y.size, 1)
model = LinearRegression()
predictions = cross_val_predict(model, X, y, cv=kf)
print ("Coefficient of Determination: %s"%r2_score(y, predictions))
print ("Root Mean Squared Error: %s"%math.sqrt(mean_squared_error(y, predictions)))
print ("Mean Absolute Error: %s"%mean_absolute_error(y, predictions))
print ("Correlation Coefficient: %s"%np.corrcoef(y, predictions, rowvar = False)[0, 1])

The ordering matters due to one specific process that you can read about [here](https://stats.stackexchange.com/questions/21022/does-the-order-of-explanatory-variables-matter-when-calculating-their-regression), but it involves some maths so be prepared.

### ========== Question 2.14 --- [4 marks] ==========
How does this more complex model perform with respect to your best performing model from either question 2.3 or 2.8? List one advantage and one disadvantage of using the more complex model.

*Previous best performing model (2.8):*
* *R^2 = 0.560579712061*
* *RMSE = 4457.03690216*
* *MAE = 3072.22425609*
* *CC = 0.7532852297*

*The more complex model performed better than the previous one based on all of the evaluation metrics:*
* *RMSE and MAE are both almost two times lower (2460.82 and 1701.36, respectively), meaning that the variance of errors this model makes is much smaller.*
* *R^2 is about 1.6 times bigger (0.866), suggesting that a big part of the variance in the price is predictable using this set of features.*
* *CC is more than 20% higher (0.933), implying that this model predictions correlate much better with the price values than the ones given by the model in question 2.8. What is more, it is high very overall (assuming that the ideal correlation is 1).*

*The main disadvantage of this model is its size (the amount of features) and complexity. Ideally, we would want to have a minimum amount of features. What is more, introducing some unimportant features also diminishes the discriminative effects of the important ones, so it would be good to perform some feature engineering and remove unhelpful features.* 

### ========== Question 2.15 --- [4 marks] ==========
Finally, experiment with tree-based regressors (e.g. [`DecisionTreeRegressor`](http://scikit-learn.org/stable/modules/generated/sklearn.tree.DecisionTreeRegressor.html), [`RandomForestRegressor`](http://scikit-learn.org/stable/modules/generated/sklearn.ensemble.RandomForestRegressor.html)) and report 5-fold cross-validation scores for R^2, RMSE, MAE and CC. Has your performance improved? Explain your answer in 1-2 sentences.

In [ ]:
regressor = DecisionTreeRegressor(random_state=0)
predictions = cross_val_predict(regressor, X, y.ravel(), cv=kf)
print ("Decision Tree Regressor results: ")
print ("Coefficient of Determination: %s"%r2_score(y, predictions))
print ("Root Mean Squared Error: %s"%math.sqrt(mean_squared_error(y, predictions)))
print ("Mean Absolute Error: %s"%mean_absolute_error(y, predictions))
print ("Correlation Coefficient: %s"%np.corrcoef(y, predictions, rowvar = False)[0, 1])

regressor = RandomForestRegressor(random_state=0, n_estimators=10)
predictions = cross_val_predict(regressor, X, y.ravel(), cv=kf)
print ("\nRandom Forest Regressor results: ")
print ("Coefficient of Determination: %s"%r2_score(y, predictions))
print ("Root Mean Squared Error: %s"%math.sqrt(mean_squared_error(y, predictions)))
print ("Mean Absolute Error: %s"%mean_absolute_error(y, predictions))
print ("Correlation Coefficient: %s"%np.corrcoef(y, predictions, rowvar = False)[0, 1])

*Neither DecisionTreeRegressor, nor RandomForestRegressor performed better than the best Multivare Linear Regression (however, both are close). The main reason could be the high-dimensionality of the data (huge amount of features), and small amount of the data points. Decision trees and random forest are not efficient in approximating high-dimensional linear relationships, because they use step functions. Their performace gets lower with the amount of features getting close to the amount of the data points. Removing unnecessary features would improve their performance.*